In [1]:
import cv2
import os
import numpy as np
import pandas as pd
import dlib
import face_recognition as fr
from datetime import datetime

In [2]:
path='images'
image_list=os.listdir(path)
image_list


['apj.jpeg',
 'mammootty.jpeg',
 'MESSI.jpg',
 'mohanlal.jpeg',
 'Pranav.jpg',
 'Sachin.jpg',
 'sehwag.jpg']

In [3]:
input_data=[]
output_data=[]
for img in image_list:
    imgpath=os.path.join(path,img)
    current_img=cv2.imread(imgpath)
    input_data.append(current_img)
    output_data.append(img.split('.')[0])

In [4]:
encoded_list=[]
def encoding(input):
    for i in input:
        conv_imgs=cv2.cvtColor(i,cv2.COLOR_BGR2RGB)
        face_coord=fr.face_locations(conv_imgs)
        encoded_list.append(fr.face_encodings(conv_imgs,face_coord)[0])
    return encoded_list

In [5]:
known_faces_encode=encoding(input_data)

read from webcam

In [6]:
attendance_rec=[]
video=cv2.VideoCapture(0)
while True:
    success,image=video.read()
    conv_img=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)

    face_loc=fr.face_locations(conv_img)
    face_encode=fr.face_encodings(conv_img,face_loc)

    for encode,faceloc in zip(face_encode,face_loc):
        matches=fr.compare_faces(known_faces_encode,encode)

        face_dist=fr.face_distance(known_faces_encode,encode)

        matching_index=np.argmin(face_dist)

        if matches[matching_index]:
            name=output_data[matching_index]
            print(name)
        else:
            name='unknown'
            print(name)

        y1,x2,y2,x1=faceloc
        cv2.rectangle(image,(x1,y1),(x2,y2),(255,0,0),2)
        cv2.putText(image,name,(x1,y1),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,0),1)

        if name != 'unknown' and name not in [i['Name'] for i in attendance_rec]:
            log={'Name':name,'Time':datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
            attendance_rec.append(log)

    cv2.imshow("Image",image)
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
video.release()
cv2.destroyAllWindows()

if attendance_rec:
    attendance_log=pd.DataFrame(attendance_rec)
    attendance_log.to_csv('D:/DL_Projects/Face recognition/attendance_log.csv',mode='a',header=False,index=False)


Pranav
Pranav
Pranav
Pranav
Pranav
